In [2]:
# Import libraries

import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, callback, dash_table, State
from datetime import datetime as dt
from datetime import timedelta
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import random
import plotly.express as px
from plotly.subplots import make_subplots
from dash import html
import os
import dash
import datetime
import subprocess
import webbrowser

# Import DATA create standard dataframe

test_data_new = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'TEST') # Import data from new database
test_data_new.dropna(how='all')
test_data_new.dropna(axis=1, how='all', inplace=True)

test_data_old= pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OLD') # Import data from old database
test_data_old.dropna(how='all')
test_data_old.dropna(axis=1, how='all', inplace=True)  

total_data = pd.concat([test_data_old, test_data_new], ignore_index=True) # Concatenate databases

cells_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'CELLS') # Import data - Cells calendar

operators_calendar = pd.read_excel('Plan.xlsx', header = 0, sheet_name = 'OPERATORS') # Import data - Operators calendar

# Declare system constants - NOT IN USE - FUTURE APPLICATIONS

cp_water = 4186 # J.(KG.°C)^(-1)
specific_mass_water = 1000 #kg/m3

cp_oil = 1880 # J.(KG.°C)^(-1)
specific_mass_oil = 900 #kg/m3

cp_glycol = 2533 # J.(KG.°C)^(-1)
specific_mass_glycol = 1040 #kg/m3

# Calculate fluid CP - NOT IN USE - FUTURE APPLICATIONS

total_data['PERC.M']= pd.to_numeric(total_data['PERC.M'], errors='coerce')
convertion_flow = 0.06 # convert flow from l/min to m3/s

total_data['CP']= 0 # Declare specific heat J.(KG.°C)^(-1)
total_data['RO']= 0 # Declare specific heat J.(KG.°C)^(-1)

total_data["RO"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', specific_mass_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', specific_mass_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*specific_mass_water) + (total_data["PERC.M"]*specific_mass_glycol))/100, 0          
                                      )))))))

total_data["CP"] = np.where(total_data["FLUIDO.CHILLER"] == 'W', cp_water, 
                            (np.where(total_data["FLUIDO.CHILLER"] == 'O', cp_oil, 
                                      (np.where(total_data["FLUIDO.CHILLER"] == 'E', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_oil))/100,
                                                (np.where(total_data["FLUIDO.CHILLER"] == 'G', (((100-total_data["PERC.M"])*cp_water) + (total_data["PERC.M"]*cp_glycol))/100, 0          
                                      ))))))) 

# Calculate cooling capacity - NOT IN USE - Future applications

convertion_flow = 0.00001666 # convert flow from l/min to m3/s

total_data['DELTA'] = (total_data['T.OUT.F.M_C']-total_data['T.IN.F.M_C']).abs() # [°C]

total_data['CALCULATED.W'] = total_data['FLUID.FLOW.L_MIN']*convertion_flow * total_data['CP'] * total_data['DELTA'] # [kW]

# SETTIME PARAMETERS AS DATETIME

total_data['STIMATION'] =  pd.to_timedelta(total_data['STIMATION'], unit='h')
total_data['DURATION'] =  pd.to_timedelta(total_data['DURATION'], unit='h')
total_data['DUE.DATE'] = pd.to_datetime(total_data['DUE.DATE'], infer_datetime_format=True)

# APP COMPONENTS

#________NAVBAR

NVent_LOGO = "https://www.nvent.com/themes/custom/particle/dist/app-drupal/assets/images/logo-nvent.svg" # Address to NVent logo

row_content = html.A(
                  dbc.Row(
                     [  
                        dbc.Col(html.Img(src=NVent_LOGO, height="30px"), style={"text-align": "left"}),
                        dbc.Col(dbc.NavbarBrand("TEXA LabData", className="ms-2") ),
                     ],
                     align="left",
                     style = {"background-color": "#E8E8E8"},
                     className="g-0",
                     ),
                    href="https://www.nvent.com/en-us/",
                    style={"textDecoration": "none", "background-color": "#E8E8E8"},
                  )

navbar = dbc.Navbar(
                dbc.Container( [
                                row_content,             
                                dbc.NavbarToggler(id="navbar-toggler", n_clicks=0)
                                ],                                     
                                fluid=True,
                                style={'display': 'flex', 'height': '40px', "background-color": "#E8E8E8"},
                                className='dashboard-nav'), 
                                style={'height': '40px', "background-color": "#E8E8E8"}
                    )

#_______DATE SELECTOR

date_card = dbc.Card(
                    [
                    dbc.CardHeader(html.H6("DATE RANGE :", className="card-title")),
                    dbc.CardBody(
                    html.Div( 
                            [
                            dcc.DatePickerRange(
                                                id="date_filter",
                                                start_date=dt(2024, 8, 1),
                                                end_date=total_data["DUE.DATE"].max(),
                                                style={
                                                        'fontSize': '12px',  # Change the font size
                                                        'width': '255px',    # Change the width of the date picker
                                                        'height': '40px'     # Change the height of the date picker
                                                        },
                                                day_size=25  # Change the size of the squares where the date is placed
                                            )
                            ]    
                            )
                        
                                )
                    ]
                    )

#______CELL SELECTOR MENU

cell_card = dbc.Card(
                    [
                    dbc.CardHeader(html.H6("STATUS :", className="card-title")),
                    dbc.CardBody(
                                [
                                dcc.Checklist(
                                        options=[
                                                {"label": "040", "value": 40},
                                                {"label": "060", "value": 60},
                                                {"label": "110", "value": 110},
                                                {"label": "Chiller", "value": "Chiller", "disabled": True},
                                                {"label": "Clima", "value": "Clima", "disabled": True}
                                                ],
                                        value=[],
                                        id = "selected_cell", 
                                            )
                                ]
                                )
                    ], className = "mx-auto"
                    )

#__________STATUS CELL SELECTOR

status_card = dbc.Card(
                        [
                        dbc.CardHeader(html.H6("CELL :", className="card-title")),
                        dbc.CardBody(
                                    [
                                    dcc.Checklist(      
                                                options=[
                                                        {"label": "COMPLETED", "value": "COMPLETED", },
                                                        {"label": "ON GOING", "value": "ON GOING"},
                                                        {"label": "HOLDING", "value": "HOLDING"},
                                                        ],
                                                value=['COMPLETED', 'HOLDING', "ON GOING"],
                                                #className="dash-bootstrap",
                                                id = "status_cell")
                                    ]
                        
                                    )
                        ], className = "mx-auto"
                        )

# ________DEFINES THE DYNAMIC TABLE FOR FILTERING THE LAB DATA
status_table = total_data[['CELL', 'TEST.ID', "P_N", "DUE.DATE", "TEST.DESCRIPTION", "TEST.STATUS", 'STIMATION', 'DURATION']]

dynamic_table = dash_table.DataTable(
                                    columns=[
                                        #{"name": i, "id": i} for i in status_table.columns
                                        {'name': 'CELL', 'id': 'CELL'},
                                        {'name': 'TEST.ID', 'id': 'TEST.ID'},
                                        {'name': 'P_N', 'id': 'P_N'},
                                        {'name': 'DUE.DATE', 'id': 'DUE.DATE'},
                                        {'name': 'TEST.DESCRIPTION', 'id': 'TEST.DESCRIPTION'},
                                        {'name': 'TEST.STATUS', 'id': 'TEST.STATUS'},
                                        {'name': 'STIMATION', 'id': 'STIMATION'},
                                        {'name': 'DURATION', 'id': 'DURATION'}
                                        ],
                                    page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '155px', 'overflowY': 'auto'},
                                    style_cell={'textAlign': 'left'},
                                    style_header = {'textAlign': 'center'},
                                    filter_action="native",
                                    sort_action="native",
                                    selected_rows=[],
                                    sort_mode='multi',
                                    row_selectable="multi",
                                    style_data_conditional=[
                                                                {
                                                                 'if': {
                                                                        'column_id': 'DUE.DATE',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '40px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'P_N',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '60px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'TEST.ID',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '40px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'TEST.STATUS',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '100px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'STIMATION',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '80px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'DURATION',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '80px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'TEST.DESCRIPTION',
                                                                                                 },
                                                                        'width': '110px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'CELL',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '40px'
                                                                }
                                                                ],
                                    id= "total_data2",
                                    data=status_table.to_dict("records") ,
                                    style_as_list_view=True                    
                                    )

table_card = dbc.Card(
        [
        dbc.CardBody(
                dynamic_table        
                    )], style={"height":"235px"}
)

#_______COUNT CELLS AVAILABILITY

total_stimation = total_data[['CELL', 'STIMATION', 'DURATION']]
date_df = total_data['DUE.DATE']

total_stimation_cell = total_stimation.groupby(['CELL']).sum().reset_index()
total_stimation_cell['STIMATION'] = total_stimation_cell['STIMATION'] / np.timedelta64(1, 'h')
total_stimation_cell['DURATION'] = total_stimation_cell['DURATION'] / np.timedelta64(1, 'h')
total_stimation_cell=total_stimation_cell.drop(total_stimation_cell[total_stimation_cell.CELL == "Chiller"].index)
total_stimation_cell=total_stimation_cell.drop(total_stimation_cell[total_stimation_cell.CELL == "Clima"].index)
dynamic_table2 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in total_stimation_cell.columns],
                                    page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    style_cell={'textAlign': 'center', 'font-size': '12px'},
                                    style_header = {'textAlign': 'center', 'font-size': '12px'},
                                    style_data_conditional=[
                                                                {
                                                                 'if': {
                                                                        'column_id': 'CELL',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '20px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'STIMATION',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '60px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'DURATION',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '60px'
                                                                }],
                                    id= "time_data",
                                    data=total_stimation_cell.to_dict("records") ,
                                    style_as_list_view=True                    
                                    )

table_card2 = dbc.Card(
        dbc.CardBody(
                dynamic_table2      
                    ), style={"height":"180px"}
)

#________CELL AVAILABILITY TABLE

cells_availability = cells_calendar.drop(['WEEKDAY'], axis = 1)
cells_availability = cells_availability.drop(['DATE'], axis = 1)
cells_availability_40 = cells_availability['40'].sum()
cells_availability_60 = cells_availability['60'].sum()
cells_availability_110 = cells_availability['110'].sum()
cells_availability_resume = pd.DataFrame({'CELL': ['40', '60', '110'],
        'AVAILABILITY': [cells_availability_40, cells_availability_60, cells_availability_110]})

dynamic_table3 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in cells_availability_resume.columns],
                                    #page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    style_cell={'textAlign': 'center', 'font-size': '12px'},
                                    style_header = {'textAlign': 'center', 'font-size': '12px'},
                                     style_data_conditional=[
                                                                {
                                                                 'if': {
                                                                        'column_id': 'CELL',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '20px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'AVAILABILITY',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '60px'
                                                                }
                                                              ],
                                    id= "time_data_cells",
                                    data=cells_availability_resume.to_dict("records"),
                                    style_as_list_view=True                       
                                    )
                

table_card3 = dbc.Card(
        dbc.CardBody(
                dynamic_table3   
                    ), style={"height":"180px"}
)

#________OPERATOR AVAILABILITY TABLE

operators_availability = operators_calendar.drop(['WEEKDAY'], axis = 1)
operators_availability = operators_availability.drop(['DATE'], axis = 1)
operators_availability_40 = operators_availability[40].sum()
operators_availability_60 = operators_availability[60].sum()
operators_availability_110 = operators_availability[110].sum()
operators_availability_resume = pd.DataFrame({'OPERATOR': ['GUERZONI', 'CARLINI', 'RESTANI'],
        'AVAILABILITY': [operators_availability_40, operators_availability_60, operators_availability_110]})

dynamic_table4 = dash_table.DataTable(
                                    columns=[{"name": i, "id": i} for i in operators_availability_resume.columns],
                                    #page_size=10,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '200px', 'overflowY': 'auto'},
                                    style_cell={'textAlign': 'center', 'font-size': '12px'},
                                    style_header = {'textAlign': 'center', 'font-size': '12px'},
                                     style_data_conditional=[
                                                                {
                                                                 'if': {
                                                                        'column_id': 'CELL',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '20px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'OPERATOR',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '60px'
                                                                }],
                                    id= "time_data_operators",
                                    data=operators_availability_resume.to_dict("records"),
                                    style_as_list_view=True                       
                                    )
                

table_card4 = dbc.Card(
        dbc.CardBody(
                dynamic_table4      
                    ), style={"height":"180px"})

#_________ELECTRICAL DATA CARD
electrical_df = pd.DataFrame({'TEST.ID', 'FASI', 'FREQUENCY.HZ','VOLTAGE.V', 'WORK.FLUID', 'FLUID.FLOW.L_MIN', 'T.IN.F.M_C','T.OUT.F.M_C','TARGET.W', 'MEASURED.W'})

dynamic_table5 = dash_table.DataTable(
                                    columns=[

                                        {'name': 'ID', 'id': 'TEST.ID'},
                                        {'name': 'FASI', 'id': 'FASI'},
                                        {'name': 'FREQUENCY(HZ)', 'id': 'FREQUENCY.HZ'},
                                        {'name': 'VOLTAGE(V)', 'id': 'VOLTAGE.V'},
                                        {'name': 'FLUID', 'id': 'WORK.FLUID'},
                                        {'name': 'FLOW(L/MIN)', 'id': 'FLUID.FLOW.L_MIN'},
                                        {'name': 'Tin(°C)', 'id': 'T.IN.F.M_C'},
                                        {'name': 'Tout(°C)', 'id': 'T.OUT.F.M_C'},
                                        {'name': 'TARGET(W)', 'id': 'TARGET.W'},
                                        {'name': 'COOLING(W)', 'id': 'MEASURED.W'}
                                       
                                        ],
                                    page_size=5,
                                    fixed_rows={'headers': True},
                                    style_table={'height': '120px', 'overflowY': 'auto'},
                                    style_cell={'textAlign': 'center', 'font-size': '10px'},
                                    style_header = {'textAlign': 'center', 'font-size': '10px'},
                                    style_data_conditional=[
                                                                {
                                                                 'if': {
                                                                        'column_id': 'TEST.ID',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '5px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'FASI',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '15px'
                                                                },
                                                                                                                                {
                                                                 'if': {
                                                                        'column_id': 'FREQUENCY.HZ',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '35px'
                                                                },
                                                                                                                                {
                                                                 'if': {
                                                                        'column_id': 'VOLTAGE.V',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '30px'
                                                                },
                                                                                                                                {
                                                                 'if': {
                                                                        'column_id': 'WORK.FLUID',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '15px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'FLUID.FLOW.L_MIN',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '30px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'T.IN.F.M_C',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '20px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'T.OUT.F.M_C',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '20px'
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'TARGET.W',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '30px',
                                                                        
                                                                },
                                                                {
                                                                 'if': {
                                                                        'column_id': 'MEASURED.W',
                                                                                                 },
                                                                        'textAlign': 'center',
                                                                        'width': '30px'
                                                                }],
                                    selected_rows=[],
                                    id= 'datatable-row-ids',
                                    data=electrical_df.to_dict("records") ,
                                    style_as_list_view=True                    
                                    )      

table_card5 = dbc.Card(
        dbc.CardBody(
                dynamic_table5,style={"height":"180px", "margin-top": "0px"}
                    )
)

#________ SEARCH ENGINE
def search_files(code):
    directories = ["V:\\TEST CHILLER\\PDF", "V:\\TEST CHILLER\\TXT"]
    results = []
    for directory in directories:
        for root, _, files in os.walk(directory):
            for file in files:
                if code in file:
                    file_path = os.path.join(root, file)
                    file_type = "report" if file.endswith(".pdf") else "log"
                    file_date = dt.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S')
                    results.append({"P/N": code, "Resultado": file, "Tipo": file_type, "Data": file_date, "Path": file_path})
    return results

app = Dash(__name__, external_stylesheets=[dbc.themes.SANDSTONE],suppress_callback_exceptions=True)


app.layout = [  dcc.Store(id='dummy_output'), dcc.Store(id='dummy_output2'), #DO NOT DELETE PDF REPORT POP UP
                dbc.Row( 
                    dbc.Col(
                        [navbar, dbc.Container(    
                                                [
                                                        dbc.Row(
                                                                    [
                                                                dbc.Col(
                                                                        [
                                                                        dbc.Row(dbc.Col(date_card),style={ "margin-top": "10px"}),
                                                                        dbc.Row(dbc.Col(cell_card),style={ "margin-top": "10px"}),
                                                                        dbc.Row(dbc.Col(status_card),style={ "margin-top": "10px"}),
                                                                        dbc.Row(dbc.Col(dbc.Card(dbc.CardBody(
                                                                           #
                                                                            dbc.Container([
                                                                                dbc.Card([
                                                                                    dbc.CardBody([
                                                                                        dcc.Input(id="input-code", type="text", placeholder="Enter 15-character code", maxLength=15),
                                                                                        dbc.Button("Search", id="search-button", color="primary", className="mt-2"),
                                                                                        dbc.Modal([
                                                                                            dbc.ModalHeader("Search Results"),
                                                                                            dbc.ModalBody(id="modal-body"),
                                                                                            dbc.ModalFooter(
                                                                                                dbc.Button("Close", id="close-modal", className="ml-auto")
                                                                                            )
                                                                                        ], id="result-modal", size="lg")
                                                                                    ])
                                                                                ], style={"width": "225px", "height": "265px", 'left-margin':'0px'}), dcc.Store(id='dummy_output3'),
                                                                                dcc.Store(id='dummy_output4')
                                                                            ])
                                                                        )
                                                                           #
                                                                            
                                                                            
                                                                            ,style={ "margin-top": "10px", "height":"300px"}) ))
                                                                        ],
                                                                        width = 2),
                                                                dbc.Col(
                                                                        [
                                                                            dbc.Row(
                                                                                        [
                                                                                    dbc.Col(table_card, width = 12)
                                                                                    
                                                                                        ],style={"height":"245px", "margin-top": "10px"}
                                                                                    ),
                                                                            dbc.Row(
                                                                                        [
                                                                                dbc.Col(
                                                                                    [
                                                                                    dbc.Row(
                                                                                        dbc.Col(
                                                                                            dbc.CardGroup([table_card3,table_card4,table_card2])
                                                                                            )
                                                                                        ,style={"height":"180px", "margin-top": "0px"}
                                                                                            ),
                                                                                    dbc.Row(
                                                                                        dbc.Col(
                                                                                            dbc.Card(
                                                                                                dbc.CardBody(dcc.Graph(id='chart1')) ,style={ "margin-top": "10px" }
                                                                                                    ), width=12
                                                                                                )
                                                                                            )
                                                                                    ] , width = 5),
                                                                                dbc.Col(
                                                                                    [
                                                                                    dbc.Row(                                                     
                                                                                        dbc.Col(
                                                                                            table_card5, width =12,style={"height":"180px", "margin-top": "0px", "width": "900px"}
                                                                                                ),style={"height":"250px", "margin-top": "0px"}
                                                                                            ),
                                                                                    dbc.Row(
                                                                                        dbc.Col(
                                                                                            html.Div(id='output_table'),style={"height":"180px", "margin-top": "0px", "width": "900px"}
                                                                                                , width=12
                                                                                                ),style={"height":"250px", "margin-top": "-60px"}
                                                                                            ),
                                                                                    dbc.Row(
                                                                                        dbc.Col(
                                                                                            html.Div(id='output_table2'),style={"height":"180px", "margin-top": "0px", "width": "900px"}
                                                                                                , width=12
                                                                                                ),style={"height":"250px", "margin-top": "-65px"}
                                                                                            )
                                                                                    ]
                                                                                        )
                                                                                        ]
                                                                                    ),
                                                                            
                                                                        ], 
                                                                        width = 10
                                                                        ),
                                                                    ],justify="center" 
                                                                ), 
                                                    ]
                                                        
                                                , 
                                        
                                        fluid=True,
                                        style={'display': 'flex'},
                                        className='dashboard-container')]
                                    )  )  ]    
#_________________________________________________________________________________________________________________________________________________________ MASTER DATA
@app.callback(   
    Output("total_data2", 'data'),
    Input('selected_cell', 'value'),
    Input('status_cell', 'value'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True
 )
            
def update_table(selected_cell, status_cell, start_date, end_date):

  
    filtered_df= status_table[ 
                                status_table["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))&
                                status_table['CELL'].isin(selected_cell)  &
                                status_table['TEST.STATUS'].isin(status_cell)
                             ].copy() 
    filtered_df['DUE.DATE'] = pd.DatetimeIndex(filtered_df['DUE.DATE']).strftime("%d/%m/%Y").copy() 
    filtered_df['STIMATION'] = filtered_df['STIMATION'] / np.timedelta64(1, 'h').copy() 
    filtered_df['DURATION'] = filtered_df['DURATION'] / np.timedelta64(1, 'h').copy() 
   

    return filtered_df.to_dict("records") 
#_________________________________________________________________________________________________________________________________________________________ CELL AVAILABILITY
@app.callback(   
    Output("time_data", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True 
 )
            
def update_table2(start_date, end_date):

    filtered_df2= total_stimation.join(date_df)
    filtered_df2 = filtered_df2[ filtered_df2["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df2= filtered_df2.drop(['DUE.DATE'], axis=1)
    filtered_df2['STIMATION'] = filtered_df2['STIMATION'] / np.timedelta64(1, 'h')
    filtered_df2['DURATION'] = filtered_df2['DURATION'] / np.timedelta64(1, 'h')
    filtered_df2=filtered_df2.groupby(['CELL']).sum().reset_index()
    filtered_df2=filtered_df2.drop(filtered_df2[filtered_df2.CELL == "Chiller"].index)
    filtered_df2=filtered_df2.drop(filtered_df2[filtered_df2.CELL == "Clima"].index)

    return  filtered_df2.to_dict("records") 
#_________________________________________________________________________________________________________________________________________________________ OPERATOR AVAILABILITY
@app.callback(   
    Output("time_data_cells", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True
 )
            
def update_table3(start_date, end_date):

    filtered_df3 = cells_calendar.drop(['WEEKDAY'], axis = 1)
    filtered_df3 = filtered_df3[ filtered_df3["DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df3= filtered_df3.drop(['DATE'], axis=1)
    filtered_availability_40 = filtered_df3['40'].sum()
    filtered_availability_60 = filtered_df3['60'].sum()
    filtered_availability_110 = filtered_df3['110'].sum()
    filtered_availability_cells = pd.DataFrame({'CELL': ['40', '60', '110'],
        'AVAILABILITY': [filtered_availability_40, filtered_availability_60, filtered_availability_110]})
 
    return filtered_availability_cells.to_dict("records")
#_________________________________________________________________________________________________________________________________________________________ STMATION
@app.callback(   
    Output("time_data_operators", 'data'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
    prevent_initial_call=True   
 )

def update_table4(start_date, end_date):

    filtered_df4 = operators_calendar.drop(['WEEKDAY'], axis = 1)
    filtered_df4 = filtered_df4[ filtered_df4["DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df4= filtered_df4.drop(['DATE'], axis=1)
    filtered_availability_operator_40 = filtered_df4[40].sum()
    filtered_availability_operator_60 = filtered_df4[60].sum()
    filtered_availability_operator_110 = filtered_df4[110].sum()
    filtered_availability_operators = pd.DataFrame({'OPERATOR': ['GUERZONI', 'CARLINI', 'RESTANI'],
        'AVAILABILITY': [filtered_availability_operator_40, filtered_availability_operator_60, filtered_availability_operator_110]})
    
    return filtered_availability_operators.to_dict("records")

#_________________________________________________________________________________________________________________________________________________________ CHART
@app.callback(   
    Output("chart1", 'figure'),
    Input('selected_cell', 'value'),
    Input("date_filter", "start_date"),
    Input("date_filter", "end_date"),
 )
            
def update_chart(selected_cell,  start_date, end_date):

    filtered_df5 = total_data[['CELL', 'TEST.ID', "P_N", "DUE.DATE", "TEST.DESCRIPTION", "TEST.STATUS", 'STIMATION', 'DURATION']]
    filtered_df5= filtered_df5[ 
                                status_table["DUE.DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))&
                                status_table['CELL'].isin(selected_cell) 
                                ]
    filtered_df5= filtered_df5.drop(['TEST.ID','P_N','DUE.DATE','TEST.DESCRIPTION','TEST.STATUS'], axis=1)
    filtered_df5['STIMATION'] = filtered_df5['STIMATION'] / np.timedelta64(1, 'h')
    filtered_df5['DURATION'] = filtered_df5['DURATION'] / np.timedelta64(1, 'h')
    filtered_df6 = filtered_df5.groupby(['CELL']).sum().reset_index()    

    filtered_df7 = cells_calendar.drop(['WEEKDAY'], axis = 1)
    filtered_df7 = filtered_df7[ filtered_df7["DATE"].between(pd.to_datetime(start_date), pd.to_datetime(end_date))]
    filtered_df7= filtered_df7.drop(['DATE'], axis=1)
    filtered_availability2_40 = filtered_df7['40'].sum()
    filtered_availability2_60 = filtered_df7['60'].sum()
    filtered_availability2_110 = filtered_df7['110'].sum()
    filtered_availability2_cells = pd.DataFrame({'CELL': ['40', '60', '110'],
        'AVAILABILITY': [filtered_availability2_40, filtered_availability2_60, filtered_availability2_110]})
    
    filtered_df6 = pd.concat([filtered_df6, filtered_availability2_cells.drop(['CELL'], axis = 1)], axis=1, join='inner')

    filtered_df6['CELL']=filtered_df6['CELL'].apply(str)
    fig = go.Figure(data=[
    go.Bar(name='DURATION', x=filtered_df6['CELL'] , y=filtered_df6['DURATION']),
    go.Bar(name='STIMATION', x=filtered_df6['CELL'] , y=filtered_df6['STIMATION']),
    go.Bar(name='AVAILABILITY', x=filtered_df6['CELL'] , y=filtered_df6['AVAILABILITY']),
    #go.Bar(name='AVAILABILITY', x=filtered_df6[filtered_df6['CELL'] == 'Value'], y=filtered_df6['AVAILABILITY'] == ['Value']),
                        ])
    fig.update_layout(title_text='Time report - Cells')
    fig.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.5,
            xanchor="center",
            x=1
        ))
    fig.update_layout(barmode='group')
    fig.update_layout(
        autosize=False,
        width=600,
        height=325,
        margin=dict(
            l=30,
            r=30,
            b=30,
            t=30,
            pad=4
        ))

    #fig.show()      

    return fig

#____________________________________________ CORRECT CODE - ELECTRICAL DATA
@callback(
    Output('datatable-row-ids', 'data', allow_duplicate = True),
    Input('total_data2', "data"),
    Input('total_data2', "selected_rows"),
    prevent_initial_call='initial_duplicate',
    allow_duplicate=True
    )

def update_result_table(rows, selected_rows):

    if not rows or not selected_rows: return[]

    selected_ids = [rows[i]['TEST.ID'] for i in selected_rows]
    electrical_df = total_data[total_data['TEST.ID'].isin(selected_ids)][['TEST.ID', 'FASI', 'FREQUENCY.HZ','VOLTAGE.V', 'WORK.FLUID', 'FLUID.FLOW.L_MIN', 'T.IN.F.M_C','T.OUT.F.M_C','TARGET.W', 'MEASURED.W']]

    return  electrical_df.to_dict("records")
#_____________________________________________
@app.callback(
    Output('output_table2', 'children'),
    Input('total_data2', 'selected_rows'),
    State('total_data2', 'data'),
    allow_duplicate=True
)
def update_output(selected_rows, data):
    if selected_rows is None:
        return 'No rows selected'

    selected_data = [data[i] for i in selected_rows]
    p_n_values2 = {i: row['P_N'].replace(' ', '') for i, row in enumerate(selected_data)}
    print(p_n_values2)

    # Directory containing the TXT files
    txt_folder = r'V:\TEST CHILLER\TXT'

    # Check if P_N values are in the names of the TXT files
    table_data2 = []
    index = 1
    for key, value in p_n_values2.items():
        matching_files2 = []
        for file_name in os.listdir(txt_folder):
            if file_name.endswith('.txt') and value in file_name.replace(' ', ''):
                file_path = os.path.join(txt_folder, file_name)
                last_modified = dt.fromtimestamp(os.path.getmtime(file_path)).strftime('%d-%m-%Y %H:%M:%S')
                matching_files2.append((file_name, last_modified))
        if matching_files2:
            for file, last_modified in matching_files2:
                table_data2.append({'Index': index, 'P_N Value': value, 'File Found': file, 'Last Modified': last_modified})
                index += 1
        else:
            table_data2.append({'Index': index, 'P_N Value': value, 'File Found': '0', 'Last Modified': 'N/A'})
            index += 1

    # Create the table
    table2 = dash_table.DataTable(id = 'result_table2',
        columns=[
            {'name': 'Index', 'id': 'Index'},
            {'name': 'P/N', 'id': 'P_N Value'},
            {'name': 'LOG File', 'id': 'File Found'},
            {'name': 'Modified in', 'id': 'Last Modified'}
        ],
        data=table_data2,
        fixed_rows={'headers': True},
        style_table={'height': '120px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'center', 'font-size': '11px'},
        style_header = {'textAlign': 'center', 'font-size': '11px'},
        row_selectable='single',
        style_as_list_view=True  
        
    )

    return dbc.Card(
        dbc.CardBody([
            table2
        ]), style={"height":"175px"}
    )
#_____________________
@app.callback(
    Output('output_table', 'children'),
    Input('total_data2', 'selected_rows'),
    State('total_data2', 'data'),
    allow_duplicate=True
)
def update_output2(selected_rows, data):
    if selected_rows is None:
        return 'No rows selected'

    selected_data = [data[i] for i in selected_rows]
    p_n_values = {i: row['P_N'].replace(' ', '') for i, row in enumerate(selected_data)}
    print(p_n_values)

    # Directory containing the TXT files
    txt_folder = r'V:\TEST CHILLER\PDF'

    # Check if P_N values are in the names of the TXT files
    table_data = []
    index = 1
    for key, value in p_n_values.items():
        matching_files = []
        for file_name in os.listdir(txt_folder):
            if file_name.endswith('.pdf') and value in file_name.replace(' ', ''):
                file_path = os.path.join(txt_folder, file_name)
                last_modified = dt.fromtimestamp(os.path.getmtime(file_path)).strftime('%d-%m-%Y %H:%M:%S')
                matching_files.append((file_name, last_modified))
        if matching_files:
            for file, last_modified in matching_files:
                table_data.append({'Index': index, 'P_N Value': value, 'File Found': file, 'Last Modified': last_modified})
                index += 1
        else:
            table_data.append({'Index': index, 'P_N Value': value, 'File Found': '0', 'Last Modified': 'N/A'})
            index += 1

    # Create the table
    table = dash_table.DataTable(id = 'result_table',
        columns=[
            {'name': 'Index', 'id': 'Index'},
            {'name': 'P/N', 'id': 'P_N Value'},
            {'name': 'Report', 'id': 'File Found'},
            {'name': 'Released in', 'id': 'Last Modified'}
        ],
        data=table_data,
        fixed_rows={'headers': True},
        style_table={'height': '120px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'center', 'font-size': '11px'},
        style_header = {'textAlign': 'center', 'font-size': '11px'},
        row_selectable='single',
        style_as_list_view=True  
        
    )

    return dbc.Card(
        dbc.CardBody([
            table
        ]), style={"height":"175px"}
    )
   
#________________________________________________
@app.callback(
    Output('dummy_output', 'data'),
    Input('result_table', 'selected_rows'),
    State('result_table', 'data')
)
def open_pdf(selected_rows, data):
    if selected_rows:
        selected_row = selected_rows[0]
        file_name = data[selected_row]['File Found']
        if file_name != '0':
            file_path = os.path.join(r'V:\TEST CHILLER\PDF', file_name)
            webbrowser.open_new(file_path)
    return ''


#___________________________________________________

@app.callback(
    Output('dummy_output2', 'data'),
    Input('result_table2', 'selected_rows'),
    State('result_table2', 'data')
)
def open_txt_in_excel(selected_rows, data):
    if selected_rows:
        selected_row = selected_rows[0]
        file_name = data[selected_row]['File Found']
        if file_name != '0':
            file_path = os.path.join(r'V:\TEST CHILLER\TXT', file_name)
            print(f"Opening file: {file_path}")  # Debugging statement

            try:
                # Read the TXT file into a DataFrame with error handling for bad lines and encoding issues
                df = pd.read_csv(file_path, delimiter='\t', encoding='latin1', on_bad_lines='skip')  # Adjust delimiter and encoding as needed

                # Save the DataFrame to an Excel file
                excel_path = file_path.replace('.txt', '.xlsx')
                df.to_excel(excel_path, index=False)

                # Open the Excel file
                if os.name == 'nt':  # Check if the OS is Windows
                    subprocess.run(['start', 'excel', excel_path], shell=True)
                else:
                    subprocess.run(['open', excel_path])
            except Exception as e:
                print(f"Error opening file: {e}")

    return ''

#___________________________________________________
@app.callback(
    Output("result-modal", "is_open"),
    Output("modal-body", "children"),
    Input("search-button", "n_clicks"),
    Input("close-modal", "n_clicks"),
    State("input-code", "value"),
    State("result-modal", "is_open")
)
def update_output3(search_clicks, close_clicks, code, is_open):
    ctx = dash.callback_context

    if not ctx.triggered:
        return is_open, ""
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == "search-button":
        if code and len(code) == 15:
            results = search_files(code)
            if results:
                df = pd.DataFrame(results)
                table = dash_table.DataTable(
                    id='result-table3',
                    columns=[{"name": i, "id": i} for i in df.columns if i != "Path"],
                    data=df.to_dict('records'),
                    row_selectable='single',
                    style_table={'overflowX': 'auto'},
                    style_cell={'textAlign': 'left'},
                    style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
                )
                return True, table
            else:
                return True, "File not found"
        else:
            return True, "Please enter a valid 15-character code"
    elif button_id == "close-modal":
        return not is_open, ""
#__________________________________________________________________________________________________________
@app.callback(
    Output('dummy_output3', 'data'),
    Input('result-table3', 'selected_rows'),
    State('result-table3', 'data')
)
def open_file3(selected_rows, data):
    if selected_rows:
        selected_row = selected_rows[0]
        file_path = data[selected_row]['Path']
        if data[selected_row]['Tipo'] == 'report':
            webbrowser.open_new(f'file:///{file_path}')
        elif data[selected_row]['Tipo'] == 'log':
            os.startfile(file_path)
    return ''

#___________________________________________________
@app.callback(
    Output('dummy_output4', 'data'),
    Input('result-table3', 'selected_rows'),
    State('result-table3', 'data')
)
def open_txt_in_excel2(selected_rows, data):
    if selected_rows:
        selected_row = selected_rows[0]
        file_path = data[selected_row]['Path']
        if data[selected_row]['Tipo'] == 'log':
            print(f"Opening file: {file_path}")  # Debugging statement

            try:
                # Read the TXT file into a DataFrame with error handling for bad lines and encoding issues
                df = pd.read_csv(file_path, delimiter='\t', encoding='latin1', on_bad_lines='skip')  # Adjust delimiter and encoding as needed

                # Save the DataFrame to an Excel file
                excel_path = file_path.replace('.txt', '.xlsx')
                df.to_excel(excel_path, index=False)

                # Open the Excel file
                if os.name == 'nt':  # Check if the OS is Windows
                    subprocess.run(['start', 'excel', excel_path], shell=True)
                else:
                    subprocess.run(['open', excel_path])
            except Exception as e:
                print(f"Error opening file: {e}")

    return ''


if __name__ == '__main__':
    app.run_server( debug=False, port=8053)
    

#debug=False,dev_tools_ui=False,dev_tools_props_check=False,


#http://127.0.0.1:8053/

c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\e2023898\AppData\Local\Temp\ipykernel_63192\3484525633.py:79: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_data['DUE.DATE'] = pd.to_datetime(total_data['DUE.DATE'], infer_datetime_format=True)


{}
{}
{}
{}
{0: 'TAOA8PMVBA10004'}
{0: 'TAOA8PMVBA10004'}
Opening file: V:\TEST CHILLER\TXT\00379999_TAOA8PMVBA10004R02_20241004_112533.txt


C:\Users\e2023898\AppData\Local\Temp\ipykernel_63192\3484525633.py:961: DtypeWarning:

Columns (321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,

{}
{}
